# Homework 2 - Data from the Web

# Task 1 - Top Universities ranking

### Importing libs

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt


import requests
from bs4 import BeautifulSoup

import json

## Extracting data from the website

In [3]:
default_path = 'https://www.topuniversities.com'
r = requests.get(default_path +'/sites/default/files/qs-rankings-data/357051.txt?_=1507995842896')
result_json = r.json()

result = pd.DataFrame.from_dict(result_json['data']).head(200)

In [4]:
for index, row in result.iterrows():
        r_ = requests.get(default_path + result.iloc[index]['url'])
        soup = BeautifulSoup(r_.text, 'html.parser')
        university = result.iloc[index]['title']
        try:
            total_faculty = soup.find('div', class_='total faculty')
            result.loc[index, 'Staff total number'] = int(total_faculty.find('div', class_='number').text.split()[0].replace(',', ''))
        except:
            print(university + ' has no values for TOTAL_STAFF') 
        try:
            total_student = soup.find('div', class_='total student')
            result.loc[index, 'Students total number'] = int(total_student.find('div', class_='number').text.split()[0].replace(',', ''))
        except:
            print(university + ' has no values for TOTAL_STUDENTS')
        try:
            inter_faculty = soup.find('div', class_='inter faculty')
            result.loc[index, 'International Staff number'] = int(inter_faculty.find('div', class_='number').text.split()[0].replace(',', ''))
        except:
            print(university + ' has no values for INTERNATIONAL_STAFF')
        try:
            inter_student = soup.find('div', class_='total inter')
            result.loc[index, 'International Students number'] = int(inter_student.find('div', class_='number').text.split()[0].replace(',', ''))
        except:
            print(university + ' has no values for INTERNATIONAL_STUDENTS')
            print(default_path + result.iloc[index]['url'])

New York University (NYU) has no values for TOTAL_STAFF
New York University (NYU) has no values for TOTAL_STUDENTS
New York University (NYU) has no values for INTERNATIONAL_STAFF
New York University (NYU) has no values for INTERNATIONAL_STUDENTS
https://www.topuniversities.com/universities/new-york-university-nyu
Indian Institute of Science (IISc) Bangalore has no values for INTERNATIONAL_STAFF


### Removing useless columns

In [5]:
result.fillna(0)
useless_columns = ['core_id','guide','nid','url','logo']
for column_name in useless_columns:
    del result[column_name]

In [6]:
df = result.copy()

## Extracting ratios

In [7]:
df['Ratio Staff/Student']=df['Staff total number']/df['Students total number']
df['Ratio International/National Students']=df['International Students number']/df['Students total number']

### Sorting dataframes by ratios

In [8]:
df_international = df.sort_values('Ratio International/National Students', ascending=0)
df_staffstud = df.sort_values('Ratio Staff/Student', ascending=0)

In [9]:
df_international.head(3)

,cc,country,rank_display,region,score,stars,title,Staff total number,Students total number,International Staff number,International Students number,Ratio Staff/Student,Ratio International/National Students
34,GB,United Kingdom,35,Europe,81.8,None,London School of Economics and Political Scien...,1088.0,9760.0,687.0,6748.0,0.111475,0.691393
11,CH,Switzerland,12,Europe,91.2,None,Ecole Polytechnique Fédérale de Lausanne (EPFL),1695.0,10343.0,1300.0,5896.0,0.163879,0.570047
7,GB,United Kingdom,8,Europe,93.7,None,Imperial College London,3930.0,16090.0,2071.0,8746.0,0.244251,0.543567


In [10]:
df_staffstud.head(3)

,cc,country,rank_display,region,score,stars,title,Staff total number,Students total number,International Staff number,International Students number,Ratio Staff/Student,Ratio International/National Students
3,US,United States,4,North America,97.7,5,California Institute of Technology (Caltech),953.0,2255.0,350.0,647.0,0.422616,0.286918
15,US,United States,16,North America,90.4,5,Yale University,4940.0,12402.0,1708.0,2469.0,0.398323,0.199081
5,GB,United Kingdom,6,Europe,95.3,5,University of Oxford,6750.0,19720.0,2964.0,7353.0,0.342292,0.372870


### Deleting useless columns before grouping by Country and Region

In [11]:

df_international_by_country_or_region = df_international.copy()
other_useless_columns = ['rank_display','score','stars','cc','Ratio International/National Students','Ratio Staff/Student','title']
for useless_column in other_useless_columns:
    del df_international_by_country_or_region[useless_column]
df_international_by_country_or_region.head(5)

,country,region,Staff total number,Students total number,International Staff number,International Students number
34,United Kingdom,Europe,1088.0,9760.0,687.0,6748.0
11,Switzerland,Europe,1695.0,10343.0,1300.0,5896.0
7,United Kingdom,Europe,3930.0,16090.0,2071.0,8746.0
198,Netherlands,Europe,1277.0,16385.0,502.0,8234.0
47,United States,North America,1342.0,13356.0,425.0,6385.0


## Grouping by country and Region

In [12]:
df_international_by_country = df_international_by_country_or_region.groupby(['country']).sum()
df_international_by_region = df_international_by_country_or_region.groupby(['region']).sum()

### Producing ratios by country

In [13]:
df_international_by_country['Ratio Staff/Student'] = df_international_by_country['Staff total number']/df_international_by_country['Students total number']
df_international_by_country['Ratio International/National Students'] = df_international_by_country['International Students number']/df_international_by_country['Students total number']

In [14]:
df_international_by_country_staffstudent = df_international_by_country.copy()
df_international_by_country_staffstudent.sort_values('Ratio Staff/Student', ascending=0).head(5)

,Staff total number,Students total number,International Staff number,International Students number,Ratio Staff/Student,Ratio International/National Students
country,,,,,,
Russia,6709.0,30233.0,373.0,5098.0,0.221910,0.168624
Denmark,11916.0,67223.0,3904.0,9543.0,0.177261,0.141960
Saudi Arabia,1062.0,6040.0,665.0,989.0,0.175828,0.163742
Singapore,9444.0,58466.0,6079.0,16168.0,0.161530,0.276537
Malaysia,2755.0,17902.0,655.0,3476.0,0.153893,0.194168


In [15]:
df_international_by_country_intstudent = df_international_by_country.copy()
df_international_by_country_intstudent.sort_values('Ratio International/National Students', ascending=0).head(5)

,Staff total number,Students total number,International Staff number,International Students number,Ratio Staff/Student,Ratio International/National Students
country,,,,,,
Australia,22034.0,301994.0,11382.0,106359.0,0.072962,0.352189
United Kingdom,79934.0,583621.0,30216.0,199426.0,0.136962,0.341705
Hong Kong,10166.0,78838.0,6296.0,24499.0,0.128948,0.310751
Austria,4117.0,63446.0,1572.0,19667.0,0.064890,0.309980
Switzerland,15323.0,109112.0,9208.0,32995.0,0.140434,0.302396


### Producing ratios by region

In [16]:
df_international_by_region['Ratio Staff/Student'] = df_international_by_region['Staff total number']/df_international_by_region['Students total number']
df_international_by_region['Ratio International/National Students'] = df_international_by_region['International Students number']/df_international_by_region['Students total number']

In [17]:
df_international_by_region_staffstudent = df_international_by_region.copy()
df_international_by_region_staffstudent.sort_values('Ratio Staff/Student', ascending=0).head(5)

,Staff total number,Students total number,International Staff number,International Students number,Ratio Staff/Student,Ratio International/National Students
region,,,,,,
Asia,106734.0,807003.0,25462.0,110100.0,0.132260,0.136431
North America,182123.0,1546353.0,43836.0,292116.0,0.117776,0.188906
Europe,218358.0,1957251.0,67598.0,449364.0,0.111564,0.229589
Latin America,45382.0,435750.0,5648.0,36871.0,0.104147,0.084615
Africa,1733.0,19593.0,379.0,3325.0,0.088450,0.169703


In [18]:
df_international_by_region_intstudent = df_international_by_region.copy()
df_international_by_region_intstudent.sort_values('Ratio International/National Students', ascending=0).head(5)

,Staff total number,Students total number,International Staff number,International Students number,Ratio Staff/Student,Ratio International/National Students
region,,,,,,
Oceania,25347.0,350167.0,12786.0,118798.0,0.072385,0.339261
Europe,218358.0,1957251.0,67598.0,449364.0,0.111564,0.229589
North America,182123.0,1546353.0,43836.0,292116.0,0.117776,0.188906
Africa,1733.0,19593.0,379.0,3325.0,0.088450,0.169703
Asia,106734.0,807003.0,25462.0,110100.0,0.132260,0.136431


## Plots and Discussion

In [19]:
#TODO: Plot your data using bar charts and describe briefly what you observed.

# Task 2 - Times Higher Education ranking

## Extracting data from the website

## Analysis

## Plots and Discussion

# Task 3 - Merging the dataframes

# Task 4 - Exploratory Analysis 

# Task 5 - Finding the best university 